In [1]:
from platform import python_version

print(python_version())

3.9.0


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pompy.models
import pompy.processors

import matplotlib.animation as animation
Writer = animation.FFMpegWriter(fps=30, codec='libx264') # Or 
Writer = animation.FFMpegWriter(fps=20, metadata=dict(artist='Me'), bitrate=1800)
%matplotlib inline
from matplotlib.animation import FuncAnimation
plt.rcParams['animation.html'] = 'html5'

#plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'

In [3]:

def set_up_figure(fig_size=(10, 5)):
    """Set up Matplotlib figure with simulation time title text.

    Parameters
    ----------
    title_text : string
        Text to set figure title to.
    fig_size : tuple
        Figure dimensions in inches in order `(width, height)`.
    """
    fig, ax = plt.subplots(1, 1, figsize=fig_size)
    title = ax.set_title('Simulation time = ---- seconds')
    return fig, ax, title

In [4]:
def update_decorator(dt, title, steps_per_frame, models):
    """Decorator for animation update methods."""
    def inner_decorator(update_function):
        def wrapped_update(i):
            for j in range(steps_per_frame):
                for model in models:
                    model.update(dt)
            t = i * steps_per_frame * dt
            title.set_text('Simulation time = {0:.3f} seconds'.format(t))
            return [title] + update_function(i)
        return wrapped_update
    return inner_decorator


In [74]:
def wind_model_demo(dt=0.001, t_max=2, steps_per_frame=20, seed=1):

    # define simulation region
    #wind_region = pompy.models.Rectangle(-0.5, 1., -1., 1.) #(0., -2., 10., 2.)

    dt = 0.005
    wind_region = pompy.models.Rectangle(x_min=0, x_max=10, y_min=-2, y_max=2)
    # set up wind model
    wind_vel_x_av =2.
    wind_vel_y_av =0.
    wind_model = pompy.models.WindModel(wind_region, 21, 11,wind_vel_x_av, wind_vel_y_av)
    # let simulation run for 10s to equilibrate wind model
    for t in np.arange(0, 2, dt):
        wind_model.update(dt)
    # generate figure and attach close event
    fig, ax, title = set_up_figure()
    # create quiver plot of initial velocity field
    vf_plot = ax.quiver(wind_model.x_points, wind_model.y_points,
                        wind_model.velocity_field.T[0],
                        wind_model.velocity_field.T[1], width=0.003)

    # expand axis limits to make vectors at boundary of field visible
    ax.axis(ax.axis() + np.array([-0.25, 0.25, -0.25, 0.25]))
    ax.set_xlabel('x-coordinate / m')
    ax.set_ylabel('y-coordinate / m')
    ax.set_aspect(1)
    fig.tight_layout()

    # define update function
    @update_decorator(dt, title, steps_per_frame, [wind_model])
    def update(i):
        vf_plot.set_UVC(
            wind_model.velocity_field.T[0], wind_model.velocity_field.T[1])
        return [vf_plot]
    t_max=2
    # create animation object
    n_frame = int(t_max / (dt * steps_per_frame) + 0.5)
    anim = FuncAnimation(fig, update, n_frame, blit=True)
    return fig, ax, anim


In [75]:
fig, ax, anim = wind_model_demo()
plt.close(fig)
anim

In [80]:
def plume_model_demo(dt=0.01, t_max=2, steps_per_frame=1, x=10., y=0.0,
                        seed=1):
    """Set up plume model and animate concentration at a point as time series.

    Demonstration of setting up plume model and processing the outputted
    puff arrays with the ConcentrationPointValueCalculator class, the
    resulting concentration time course at a point in the odour plume being
    displayed with the Matplotlib `plot` function.

    Parameters
    ----------
    dt : float
        Simulation timestep.
    t_max : float
        End time to simulate to.
    steps_per_frame: integer
        Number of simulation time steps to perform between animation frames.
    x : float
        x-coordinate of point to measure concentration at.
    y : float
        y-coordinate of point to measure concentration at.
    seed : integer
        Seed for random number generator.

    Returns
    -------
    fig : Figure
        Matplotlib figure object.
    ax : AxesSubplot
        Matplotlib axis object.
    anim : FuncAnimation
        Matplotlib animation object.
    """
    rng = np.random.RandomState(seed)
    # define simulation region
    wind_vel_x_av =2.
    wind_vel_y_av =0.
    centre_rel_diff_scale = 1.5
    puff_release_rate=100
    puff_init_rad=0.001
    source_location=(0,0)
    
    wind_region = pompy.models.Rectangle(x_min=0, x_max=10., y_min=-2, y_max=2.)
    sim_region = pompy.models.Rectangle(x_min=-0.5, x_max=10, y_min=-2, y_max=2)
    # set up wind model
    wind_model = pompy.models.WindModel(wind_region, 21, 11,wind_vel_x_av, wind_vel_y_av )
    # set up plume model
    plume_model = pompy.models.PlumeModel(
        wind_region, (source_location[0], source_location[1], 0.), wind_model, centre_rel_diff_scale=centre_rel_diff_scale,
            puff_release_rate=puff_release_rate, 
            puff_init_rad=puff_init_rad,)
    

        
    # let simulation run for 10s to initialise models
#     for t in np.arange(0, 2, dt):
#         wind_model.update(dt)
#         plume_model.update(dt)
    # set up concentration point value calculator
    fig, ax, title = set_up_figure()
    
    vf_plot = ax.quiver(wind_model.x_points, wind_model.y_points,
                        wind_model.velocity_field.T[0],
                        wind_model.velocity_field.T[1], width=0.003)
    
    
    ax.axis(ax.axis() + np.array([-0.25, 0.25, -0.25, 0.25]))
    
    radius_mult = 200
    pp_plot = plt.scatter(
        plume_model.puff_array[:, 0], plume_model.puff_array[:, 1],
        radius_mult * plume_model.puff_array[:, 3]**0.5, c='r',
        edgecolors='none')
    
    ax.set_xlabel('x-coordinate / m')
    ax.set_ylabel('y-coordinate / m')
    ax.set_aspect(1)
    fig.tight_layout()

    # define update function
    @update_decorator(dt, title, steps_per_frame, [wind_model, plume_model])
    def update(i):
        # update velocity field quiver plot data
        vf_plot.set_UVC(wind_model.velocity_field[:, :, 0].T,
                        wind_model.velocity_field[:, :, 1].T)
        # update puff position scatter plot positions and sizes
        print(plume_model.puff_array.shape)
        pp_plot.set_offsets(plume_model.puff_array[:, :2])
        pp_plot._sizes = radius_mult * plume_model.puff_array[:, 3]**0.5
        print(pp_plot._sizes)
        return [vf_plot, pp_plot]

    # create animation object
    n_frame = int(t_max / (dt * steps_per_frame) + 0.5)
    anim = FuncAnimation(fig, update, frames=n_frame, blit=True)
    return fig, ax, anim

    
    

In [81]:
fig, ax, anim = plume_model_demo()
plt.close(fig)
anim

(11, 4)
[0.66332496 0.66332496 0.66332496 0.66332496 0.66332496 0.66332496
 0.66332496 0.66332496 0.66332496 0.66332496 0.66332496]
(11, 4)
[0.91651514 0.91651514 0.91651514 0.91651514 0.91651514 0.91651514
 0.91651514 0.91651514 0.91651514 0.91651514 0.91651514]
(12, 4)
[1.11355287 1.11355287 1.11355287 1.11355287 1.11355287 1.11355287
 1.11355287 1.11355287 1.11355287 1.11355287 1.11355287 0.66332496]
(12, 4)
[1.28062485 1.28062485 1.28062485 1.28062485 1.28062485 1.28062485
 1.28062485 1.28062485 1.28062485 1.28062485 1.28062485 0.91651514]
(13, 4)
[1.42828569 1.42828569 1.42828569 1.42828569 1.42828569 1.42828569
 1.42828569 1.42828569 1.42828569 1.42828569 1.42828569 1.11355287
 0.66332496]
(14, 4)
[1.56204994 1.56204994 1.56204994 1.56204994 1.56204994 1.56204994
 1.56204994 1.56204994 1.56204994 1.56204994 1.56204994 1.28062485
 0.91651514 0.66332496]
(14, 4)
[1.68522995 1.68522995 1.68522995 1.68522995 1.68522995 1.68522995
 1.68522995 1.68522995 1.68522995 1.68522995 1.6852299

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [15]:
def conc_point_val_demo(dt=0.01, t_max=5, steps_per_frame=1, x=10., y=0.0,
                        seed=1):
    """Set up plume model and animate concentration at a point as time series.

    Demonstration of setting up plume model and processing the outputted
    puff arrays with the ConcentrationPointValueCalculator class, the
    resulting concentration time course at a point in the odour plume being
    displayed with the Matplotlib `plot` function.

    Parameters
    ----------
    dt : float
        Simulation timestep.
    t_max : float
        End time to simulate to.
    steps_per_frame: integer
        Number of simulation time steps to perform between animation frames.
    x : float
        x-coordinate of point to measure concentration at.
    y : float
        y-coordinate of point to measure concentration at.
    seed : integer
        Seed for random number generator.

    Returns
    -------
    fig : Figure
        Matplotlib figure object.
    ax : AxesSubplot
        Matplotlib axis object.
    anim : FuncAnimation
        Matplotlib animation object.
    """
    rng = np.random.RandomState(seed)
    # define simulation region
    sim_region = pompy.models.Rectangle(x_min=0., x_max=100, y_min=-25., y_max=25.)
    # set up wind model
    wind_model = pompy.models.WindModel(sim_region, 21, 11, rng=rng)
    # set up plume model
    plume_model = pompy.models.PlumeModel(
        sim_region, (0., 0., 0.), wind_model, rng=rng)
    # let simulation run for 10s to initialise models
    for t in np.arange(0, 2, dt):
        wind_model.update(dt)
        wind_model.update(dt)
        plume_model.update(dt)
    # set up concentration point value calculator
    
    val_calc = pompy.processors.ConcentrationValueCalculator(1.)
    conc_vals = []
    conc_vals.append(val_calc.calc_conc_point(plume_model.puff_array, x, y))
    ts = [0.]
    # set up figure
    fig, ax, title = set_up_figure()
    # display initial concentration field as image
    conc_line, = plt.plot(ts, conc_vals)
    ax.set_xlim(0., t_max)
    ax.set_ylim(0., 150.)
    ax.set_xlabel('Time / s')
    ax.set_ylabel('Normalised concentration')
    ax.grid(True)
    fig.tight_layout()

    # define update function
    @update_decorator(dt, title, steps_per_frame, [wind_model, plume_model])
    def update(i):
        ts.append(dt * i * steps_per_frame)
        conc_vals.append(
            val_calc.calc_conc_point(plume_model.puff_array, x, y))
        conc_line.set_data(ts, conc_vals)
        return [conc_line]

    # create animation object
    n_frame = int(t_max / (dt * steps_per_frame) + 0.5)
    anim = FuncAnimation(fig, update, frames=n_frame, blit=True)
    return fig, ax, anim

In [16]:
fig, ax, anim = plume_model_demo()
plt.close(fig)
anim